# Import Libraries

In [ ]:
!pip install --upgrade autogluon

In [ ]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularPredictor

import os
import gc
train_path = '../input/tabular-playground-series-jun-2021/train.csv'
test_path = '../input/tabular-playground-series-jun-2021/test.csv'
RS = 69420
# os.environ['NUMEXPR_MAX_THREADS'] = '16'

# Preprocessing

In [ ]:
df = pd.read_csv(train_path, index_col=0)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['target'] = le.fit_transform(df['target'])

df

In [ ]:
gc.collect()

In [ ]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_jobs=-1, bootstrap=True,verbose=1)

In [ ]:
%%time
clf = clf.fit(X, y)

In [ ]:
from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(clf, prefit=True)
X = model.transform(X)
X.shape

In [ ]:
df = pd.DataFrame(X)
df['target'] = y

In [ ]:
df

In [ ]:
del X, y
gc.collect()

# AutoGluon Model

In [ ]:
label = 'target'
eval_metric='log_loss'
save_path = './AutoGlon/'
timeLimit = 3600 * 7.5
num_trials = 250
search_strategy = 'auto' 

In [ ]:
hyperparameter_kwargs = {'num_trials': num_trials,
                         'scheduler' : 'local',
                         'searcher': search_strategy
                        }

In [ ]:
models={'NN': {},
        'GBM': [{'device':'gpu', 'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},{'device':'gpu'},'GBMLarge',],
        'CAT': {'task_type':'GPU'},
        'XGB': {'tree_method':'gpu_hist'},
        'FASTAI': {}}

In [ ]:
predictor = TabularPredictor(label=label,
                             eval_metric=eval_metric,
                             path=save_path,
                             verbosity=1) # 0 = Silent, 4 = Print every damn thing

In [ ]:
%%time
predictor.fit(df,
              hyperparameters=models,
              presets='best_quality',
              keep_only_best=True,
              refit_full=True,
              save_space=True,
              hyperparameter_tune_kwargs=hyperparameter_kwargs,
              time_limit=timeLimit)

results = predictor.fit_summary()

# Predict Model

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv', index_col=0)
submission

In [ ]:
test = pd.read_csv(test_path, index_col=0)
test

In [ ]:
test = model.transform(test)

In [ ]:
y_preds = predictor.predict_proba(test)

In [ ]:
preds = y_preds.values

submission['Class_1']=preds[:,0]
submission['Class_2']=preds[:,1]
submission['Class_3']=preds[:,2]
submission['Class_4']=preds[:,3]
submission['Class_5']=preds[:,4]
submission['Class_6']=preds[:,5]
submission['Class_7']=preds[:,6]
submission['Class_8']=preds[:,7]
submission['Class_9']=preds[:,8]
submission.head()

In [ ]:
submission.to_csv('submission.csv')